In [6]:
#@title Salary Cap Game
import pandas as pd
import matplotlib.pylab as plt
%pylab inline

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from IPython.display import clear_output

###########################################################################################################################

salaries = pd.read_csv('salaries.csv')
salaries = pd.DataFrame(salaries.drop_duplicates(subset='Player', keep='first'))
salaries['Rk'] = list(range(1, len(salaries) + 1))
salaries['2018-19'] = salaries['2018-19'].apply(lambda x: x.replace('$', ''))
salaries['2018-19'] = salaries['2018-19'].apply(lambda x: x.replace(',', ''))
salaries['2018-19'] = pd.to_numeric(salaries["2018-19"])
salaries.index = salaries['Rk']

###########################################################################################################################

#average_payroll = salaries.pivot_table(index=['Tm'], aggfunc=sum)[['2018-19']].mean().iloc[0]
#average_payroll = round(average_payroll)
average_payroll = 100000000

players = []
for x in range(1, len(salaries['Player']) + 1):
    players.append(salaries['Player'][x])

teams = []
def make_teams():
    global teams
    teams = ["All Teams"]
    for z in range(0, len(salaries.pivot_table(index=['Tm']).index)):
        teams.append(salaries.pivot_table(index=['Tm']).index[z])
make_teams()

Populating the interactive namespace from numpy and matplotlib


In [7]:
roster = []
cash_roster = []
current_player = ""
selected_from_roster = ""
selected_team = ""

roster_df = salaries.loc[cash_roster][['Player', 'Tm', '2018-19']].sort_values(by=['2018-19'], ascending=False)
your_payroll = 0
cash_left = average_payroll

In [8]:
def number_string(number):
    currency = '${:,.0f}'.format(number)
    return currency

def cash_display_int(df):
    df = df.sort_values(by=['2018-19'], ascending=False)
    df['2018-19'] = df['2018-19'].astype(int)
    df['2018-19'] = df['2018-19'].apply(number_string)

    df.index = list(range(1, len(df) + 1))
    display(df)

    df['2018-19'] = df['2018-19'].apply(lambda x: x.replace('$', ''))
    df['2018-19'] = df['2018-19'].apply(lambda x: x.replace(',', ''))
    df['2018-19'] = df['2018-19'].astype(int)
   
def introduction():
    print("You have 12 roster spots to construct your own NBA team.")
    print("You must stay under the salary cap:", number_string(average_payroll), ".")

def dropdown_picks():
    global selected_team
    global teams
    global your_payroll
    global cash_left
    
    remaining_teams = list(salaries[(salaries['2018-19'] <= cash_left)].pivot_table(index=['Tm']).index)
    teams = ["All Teams"] + remaining_teams
    
    if selected_team == "":
        pass
    else:
        if selected_team in remaining_teams:
            teams.remove(selected_team)
            teams.insert(0, selected_team)
        else:
            pass
    
    interact(choose_team, Team=teams)
    display(button_add)
    
def initial_salary():
    print("You have spent: $0")
    print("You have", number_string(average_payroll), "left.")

def spent_left():
    global roster
    global roster_df
    global your_payroll
    global cash_left
    
    print("You have spent:", number_string(your_payroll))
    print("You have", number_string(cash_left), "left.")

def display_roster():
    if roster == []:
        pass
    else:
        cash_display_int(roster_df)
    
#SECTION
def delete_section():
    display(button_delete)
    your_roster = list(roster_df.sort_values(by='2018-19', ascending=False)['Player'])
    interact(select_deleted, Your_Roster=your_roster)
    
#SECTION
def reset_section():
    display(button_reset)
    
#DROPDOWN
def choose_team(Team):
    global roster
    global current_player
    global selected_team
    global your_payroll
    global cash_left
    
    team = Team
    selected_team = Team
    dropdown_list = salaries[~salaries['Rk'].isin(cash_roster) & (salaries['2018-19'] <= cash_left)]
    
    if dropdown_list.empty == True:
        current_player = ""
        interact(choose_player, Player=[])
    else:
        if team == 'All Teams':
            interact(choose_player, Player=dropdown_list['Player'])
        else:
            interact(choose_player, Player=dropdown_list[(dropdown_list['Tm'] == team)]['Player'])
    
#DROPDOWN
def choose_player(Player):
    global current_player
    player = Player
    
    if player == None:
        current_player = ""
        print("You don't have enough cap space to sign any remaining players from this team.")
    else:
        salary = salaries[(salaries['Player'] == player)]['2018-19'].iloc[0]
        current_player = player
    
        print("Salary:", number_string(salary))

#DROPDOWN
def select_deleted(Your_Roster):
    global selected_from_roster
    selected_from_roster = Your_Roster

#BUTTON
def add_player (c):
    global test_works
    global current_player
    global roster_df
    global salaries
    global selected_team
    global teams
    global your_payroll
    global cash_left
    
    if current_player == "":
        pass
    else:
        roster.append(current_player)
        cash_roster.append( salaries[(salaries['Player'] == current_player)]['Rk'].iloc[0] )

        current_team = salaries[(salaries['Player'] == current_player)]['Tm'].iloc[0]
        current_salary = salaries[(salaries['Player'] == current_player)]['2018-19'].iloc[0]
        
        roster_df = roster_df.append({"Player": current_player, "Tm": current_team, "2018-19": current_salary}, ignore_index=True)
        your_payroll = roster_df[['2018-19']].sum().iloc[0]
        cash_left = average_payroll - your_payroll

        clear_output()
        introduction()
        dropdown_picks()
        spent_left()
        display_roster()
        reset_section()
        delete_section()
        
#BUTTON
def delete_player(d):
    global selected_from_roster
    global roster_df
    global roster
    global your_payroll
    global cash_left
    
    if roster == []:
        pass
    else:
        selected_index = roster_df[(roster_df['Player'] == selected_from_roster)].index[0]
        roster_df = roster_df.drop([selected_index])
        your_payroll = roster_df[['2018-19']].sum().iloc[0]
        cash_left = average_payroll - your_payroll

        roster.remove(selected_from_roster)
        cash_roster.remove(salaries[(salaries['Player'] == selected_from_roster)]['Rk'].iloc[0])
            
        if roster == []:
            clear_output()
            introduction()
            dropdown_picks()
            spent_left()
            display_roster()
        else:
            clear_output()
            introduction()
            dropdown_picks()
            spent_left()
            display_roster()
            reset_section()
            delete_section()

#BUTTON
def reset_roster(r):
    global roster
    global cash_roster
    global roster_df
    global your_payroll
    global cash_left
    
    roster = []
    cash_roster = []
    
    roster_df = roster_df.iloc[0:0]
    your_payroll = 0
    cash_left = average_payroll
    
    make_teams()
    
    clear_output()
    introduction()
    dropdown_picks()
    initial_salary()
    
#APPLICATION
def application():
    global roster_df
    
    introduction()
    dropdown_picks()
    spent_left()
    display_roster()

    if len(roster_df) == 0:
        pass
    elif len(roster_df) > 0:
        reset_section()
        delete_section()

In [9]:
button_add = widgets.Button(description="Add Player")
button_add.on_click(add_player)

button_delete = widgets.Button(description="Delete Player Below")
button_delete.on_click(delete_player) 

button_reset = widgets.Button(description="Reset Roster")
button_reset.on_click(reset_roster) 

To play the game, select "Runtime" in the navigation bar above and click "Run all.

<br>

You can also manually load the game by: 

1) clicking the triangle to the left of where it says "Salary Cap Game" above

2) clicking the triangle to the left of where it says "application()" below.

In [10]:
application()

You have 12 roster spots to construct your own NBA team.
You must stay under the salary cap: $100,000,000 .


interactive(children=(Dropdown(description='Team', options=('All Teams', 'ATL', 'BOS', 'BRK', 'CHI', 'CHO', 'C…

Button(description='Add Player', style=ButtonStyle())

You have spent: $0
You have $100,000,000 left.
